In [ ]:
#debut de la run
#charge les données
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms
import numpy as np

class PKLVideoDataset(Dataset):
    def __init__(self, x_files, y_files, transform=None, target_frames=16):
        self.videos = []
        self.labels = []
        self.transform = transform
        self.target_frames = target_frames

        # Charger tous les fichiers Pickle
        for xf, yf in zip(x_files, y_files):
            with open(xf, "rb") as f:
                self.videos.extend(pickle.load(f))
            with open(yf, "rb") as f:
                self.labels.extend(pickle.load(f))

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video = self.videos[idx]
        label = self.labels[idx]

        # Convertir BGR → RGB
        frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in video]

        # Ajuster à target_frames
        if len(frames) < self.target_frames:
            repeat_factor = int(np.ceil(self.target_frames / len(frames)))
            frames = (frames * repeat_factor)[:self.target_frames]
        else:
            frames = frames[:self.target_frames]

        # Appliquer transform
        if self.transform:
            frames = [self.transform(f) for f in frames]

        # Empiler : (C, T, H, W)
        video_tensor = torch.stack(frames, dim=1)
        return video_tensor, torch.tensor(label, dtype=torch.long)

#les fichiers 7 sont pour la validation
x_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train8_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val8_small3.pkl",

]
y_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train8.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val8.pkl",

]
# Fichiers de validation
x_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train7_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val7_small3.pkl",

]
y_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train7.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val7.pkl",

]


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                         std=[0.22803, 0.22145, 0.216989]),
])
from google.colab import drive

# ============================================================
# 2️⃣ MONTAGE DU DRIVE ET CHARGEMENT DU DATAFRAME
# ============================================================
drive.mount("/content/drive", force_remount=True)
train_dataset = PKLVideoDataset(x_files, y_files, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

# Création du Dataset et DataLoader pour validation
val_dataset = PKLVideoDataset(x_val_files, y_val_files, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

Mounted at /content/drive


In [ ]:
print(len(train_dataset))
print(len(val_dataset))

6509
497


In [ ]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(42)

In [ ]:
#entrainement du modèle
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler
class R2Plus1D_Custom(nn.Module):
    def __init__(self, num_classes=20, dropout_p=0.5):
        super().__init__()
        # Charger le backbone pré-entraîné
        self.backbone = models.video.r2plus1d_18(
            weights=models.video.R2Plus1D_18_Weights.KINETICS400_V1
        )
        in_features = self.backbone.fc.in_features

        # Remplacer la tête de classification
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_p),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

# ============================================================
# 🔹 ENTRAÎNEMENT
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = R2Plus1D_Custom(num_classes=20, dropout_p=0.3).to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()
scaler = GradScaler()

model_path = "/content/drive/MyDrive/Projet non-alternant/Vidéo/new_r2.pth"
num_epoch = 10
best_acc = 0.0
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epoch)
train_acc_list, val_acc_list, train_loss_list, val_loss_list = [], [], [], []

for epoch in range(num_epoch):
    model.train()
    running_train_loss, running_train_correct, total_train = 0.0, 0, 0

    for videos, labels in tqdm(train_loader, desc=f"Époque {epoch+1}/{num_epoch}"):
        videos, labels = videos.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(videos)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_train_loss += loss.item()
        preds = outputs.argmax(1)
        running_train_correct += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_train_loss / len(train_loader)
    train_acc = running_train_correct / total_train

    # ---- Validation ----
    model.eval()
    running_val_loss, running_val_correct, total_val = 0.0, 0, 0
    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(device), labels.to(device)
            with autocast():
                outputs = model(videos)
                loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            preds = outputs.argmax(1)
            running_val_correct += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_loader)
    val_acc = running_val_correct / total_val

    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    print(f"📊 [Époque {epoch+1}/{num_epoch}] "
          f"Train Acc: {train_acc:.4f} | Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")
    scheduler.step()
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"💾 Nouveau meilleur modèle sauvegardé avec Val Acc = {best_acc:.4f}")

Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


100%|██████████| 120M/120M [00:00<00:00, 192MB/s]
/tmp/ipython-input-826655661.py:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Époque 1/20:   0%|          | 0/814 [00:00<?, ?it/s]/tmp/ipython-input-826655661.py:65: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Époque 1/20: 100%|██████████| 814/814 [04:10<00:00,  3.24it/s]
/tmp/ipython-input-826655661.py:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


📊 [Époque 1/20] Train Acc: 0.2334 | Train Loss: 2.6611 | Val Acc: 0.3662 | Val Loss: 2.2606
💾 Nouveau meilleur modèle sauvegardé avec Val Acc = 0.3662


Époque 2/20: 100%|██████████| 814/814 [04:13<00:00,  3.21it/s]


📊 [Époque 2/20] Train Acc: 0.3799 | Train Loss: 2.1876 | Val Acc: 0.4024 | Val Loss: 2.0204
💾 Nouveau meilleur modèle sauvegardé avec Val Acc = 0.4024


...

📊 [Époque 7/20] Train Acc: 0.7135 | Train Loss: 1.0541 | Val Acc: 0.4588 | Val Loss: 1.9476
💾 Nouveau meilleur modèle sauvegardé avec Val Acc = 0.4588


Époque 8/20: 100%|██████████| 814/814 [04:13<00:00,  3.21it/s]


📊 [Époque 8/20] Train Acc: 0.7723 | Train Loss: 0.8736 | Val Acc: 0.4527 | Val Loss: 1.9591


Époque 9/20:   3%|▎         | 25/814 [00:08<04:21,  3.01it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, num_epoch + 1)

plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
# Calculate accuracy for each epoch
train_acc_epochs = [acc for acc in train_acc_list]
val_acc_epochs = [acc for acc in val_acc_list]

plt.plot(epochs, train_acc_epochs, 'b-o', label='Train Acc')
plt.plot(epochs, val_acc_epochs, 'r-o', label='Val Acc')
plt.title('Courbe de précision')
plt.xlabel('Époque')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)



In [ ]:
# Loss
plt.subplot(1,2,2)
# Calculate average loss per batch for each epoch
train_loss_epochs = [loss for loss in train_loss_list]
val_loss_epochs = [loss  for loss in val_loss_list]

plt.plot(epochs, train_loss_epochs, 'b-o', label='Train Loss')
plt.plot(epochs, val_loss_epochs, 'r-o', label='Val Loss')
plt.title('Courbe de perte')
plt.xlabel('Époque')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()